## Notes and ideas :
- look for the frequency of publication of articles.
#### Features to be included in the template:
- Images with more weighting than the body of the article ( they may contain more information and impact on the reader).
- Measure the degree of objectivity or subjectivity (using the Vader metric) of bbc news writers by calculating the average distance of a bbc article from neutralitypoint (this could be done by applying the prediction model on more various categories -hence more data injected) , input this into your main model.
- Count the number of authors and see if studying each author separately is efficient, knowing that no much writters write in the climate rubrique.
- A picture (if subjective) is worth a thousand words", but if it is not significant, i.e. objective, it has no impact, no matter how much we consider it, the final result will not be affected. so we multiply the picture text (not a thousand times ofc hhh) to ponder with the rest of the article content.

## Insight :
- Transfer learning is already applied, make sure in the insight that you offer a path to relearn the Vader program to articles similar to bbc news.  

In [64]:
import requests
import json
import numpy as np
import pandas as pd
import matplotlib
from newsapi import NewsApiClient
from bs4 import BeautifulSoup
import requests
import csv
import os
import urllib.request
from tqdm import *

# I- Fetching data :

Used to fetch news data using an original scrapper, including published images, so they be injected into the model as additional features  

In [65]:
def scrap_one_article(url,_,nbr_articles):
    try:
        dirName = 'images'
        os.mkdir(dirName)
    except:
        pass
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    csv_file = open('bbc_news.csv', 'a')

    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['headline', 'summary', 'image_link', 'category'])
    i = 98
    try:
        title = soup.find_all(class_ = 'gs-c-promo-heading-title')[0].text
        Text = article.find(class_='gs-c-promo-summary').text
        category = article.find(class_='gs-c-section-link').text.strip()
    except Exception as e:
        category = 'None'

    try:
        image_tag = str(article.find('img'))
        elements = image_tag.split('src="')
        image_link = elements[1].split('"')[0]
    except Exception as e:
        image_link = None
    Content_list = soup.find_all(class_ = 'eq5iqo00')
    article_content = build_text(Content_list)
    print('fetched the name of author :')
    print("fetched the article's content : ",article_content[:100],f' +[{len(article_content)-100}] ',)
    article_dic= {'id': _ ,'author':None,'article_content':article_content}
    
    download_images(soup,_,nbr_articles)
    return article_dic

In [66]:
def build_text(list_):
    list_.pop()
    return '\n'.join([x.text for x in list_])
def dl_img(url, file_path, file_name):
    full_path = file_path + file_name + '.jpg'
    urllib.request.urlretrieve(url, full_path)
    return full_path
def download_images(soup_result,article_index,nbr_articles):
    nbr_img = 0
    try:
        os.mkdir(f'images/Article{article_index}')
    except:
        pass
    img_results = [res['src'] for res in soup_result.find_all('img')]
    for i in tqdm(range(len(img_results)),desc = f'Downloading images /{nbr_articles}'):
        image_link = img_results[i]
        image_link_loc = f'images/Article{article_index}/'
        file_name = f'img{i}'
        try:
            image_link_local = dl_img(image_link,image_link_loc, file_name)
            i += 1
            nbr_img+=1
        except Exception as identifier:
            image_link_local = "EXTERNAL: " + image_link
            i += 1
    print('\nArticle',article_index,"'s",nbr_img,' images were downloaded succesfully, ',nbr_articles,'articles remains')
def get_links_list(soup):
        results = soup.find_all('a',class_ = 'gs-c-promo-heading')
        links_list = ['http://www.bbc.com/'+res['href'] for res in results]
        return links_list

In [80]:
def scrap_bbc_articles(category_url):
    url = category_url
    response = requests.get(url)
    print('Request status (200 means a succesful request): ',response.status_code)
    soup = BeautifulSoup(response.text, 'lxml')
    csv_file = open('bbc_news.csv', 'a')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['id', 'title', 'publishing date', 'body'])
    links = get_links_list(soup)
    print('Fetching data from ',len(links) ,'bbc news climate articles')
    results = []
    for _ in range(1,len(links)):  # Covers all the articles len(links)+1
        print('------------------------------------')
        try: 
            A = scrap_one_article(links[_],_,len(links))
            results.append(A)
        except: ValueError
    return results


In [81]:
climate_url = 'https://www.bbc.com/news/science-environment-56837908'
data = scrap_bbc_articles(climate_url)
def fetch_clean_data(data_list): 
    article_results = [] 
    for i in range(len(data_list)):
        try:
            article_dict = {}
            article_dict['id'] = data_list[i]['id']
            article_dict['author'] = data_list[i]['author']
            article_dict['article_content'] = data_list[i]['article_content'].strip("\n").strip("\t")
            article_results.append(article_dict)
        except IndexError:
            pass
    return article_results

Request status (200 means a succesful request):  200
Fetching data from  43 bbc news climate articles
------------------------------------


fetched the name of author :
fetched the article's content :  When eight teenagers and an elderly nun in Australia teamed up for a climate case, they won, in a hi  +[7903] 



Article 1 's 17  images were downloaded succesfully,  43 articles remains
------------------------------------


fetched the name of author :
fetched the article's content :  The highest temperature ever recorded in the Arctic, 38C (100F), has been officially confirmed, soun  +[2977] 



Article 2 's 13  images were downloaded succesfully,  43 articles remains
------------------------------------


...

fetched the name of author :
fetched the article's content :  In our monthly feature, Then and Now, we reveal some of the ways that planet Earth has been changing  +[5733] 



Article 34 's 17  images were downloaded succesfully,  43 articles remains
------------------------------------


fetched the name of author :
fetched the article's content :  In our monthly feature, Then and Now, we reveal some of the ways that planet Earth has been changing  +[3649] 



Article 35 's 16  images were downloaded succesfully,  43 articles remains
------------------------------------


------------------------------------
------------------------------------
------------------------------------
------------------------------------
------------------------------------
------------------------------------


In [87]:
Data = pd.DataFrame.from_dict(fetch_clean_data(data))
pd.set_option('max_colwidth', 500)


Data.head()

,id,author,article_content
0,1,None,"When eight teenagers and an elderly nun in Australia teamed up for a climate case, they won, in a historic judgement. Their case has now been appealed by the country's government. If the final verdict swings in their favour, it will have ramifications not just for Australian law but for climate cases world-wide. \nIn May this year, Anjali Sharma was sitting in her economics class at school in Melbourne when the court in Sydney live-streamed the results of a climate case she had found herself..."
1,2,None,"The highest temperature ever recorded in the Arctic, 38C (100F), has been officially confirmed, sounding ""alarm bells"" over Earth's changing climate.\nThe World Meteorological Organization (WMO) on Tuesday verified the record, reported in the Siberian town of Verkhoyansk on 20 June last year. \nThe temperature was 18C higher than the area's average daily maximum for June.\nThe WMO, a UN agency, said the extreme heat was ""more befitting the Mediterranean than the Arctic"".\nIt is the first tim..."
2,3,None,"Scientists are warning of dramatic changes at one of the biggest glaciers in Antarctica, potentially within the next five to 10 years.\nThey say a floating section at the front of Thwaites Glacier that until now has been relatively stable could ""shatter like a car windscreen"". \nUS and UK researchers are currently engaged in an intense study programme at Thwaites because of its melt rate. \nAlready it is dumping 50 billion tonnes of ice into the ocean each year.\nThis is having limited impac..."
3,4,None,"Making your home more energy efficient can be costly and may not be possible if you're renting, or you don't have thousands of pounds stashed away to buy new heat pumps and double glazing.\nBut there are some cheap solutions to try to save money. We've looked at where you are likely to be losing the most energy in your home and come up with some simple solutions to help save money on your bills and keep warm this winter.\nWarm air wants to leave your home and will find any nook and cranny to..."
4,5,None,"Indonesia pledged at the recent COP26 climate summit that its greenhouse gas emissions would peak by 2030 and then start to fall. \nIt's also said that it will end deforestation by that same date.\nBut to reduce emissions from its transport sector, it's relying on using more biofuels - production of which can lead to the loss of forested land.\nSo how can it both curb its emissions using biofuels and end deforestation by 2030?\nIndonesia is now the third largest producer of biofuels in the w..."


In [85]:
Data.to_excel('exported_bbc_data.xlsx')

<h1> II- Data preprocessing :</h1>

### Introduction
Feature engineering is an important process in the pipeline of any machine learning project.
It involves applying a series of transformations to the data, including
data, including eliminating bad records, encoding variables, scaling variables and finally eliminating
scaling of variables and finally the elimination of variables with a high correlation.
The motivation behind this process is to feed the algorithms with digested data
and to improve their performance by reducing the learning time.
In this project, different data processing techniques were applied to implement the machine learning models.
the machine learning models and improve their performance, which we will discuss below.
we will discuss below.
### Missing value rate and zero variance
Here we try to eliminate variables with a very high rate of missing values
or zero variance. The following variables are then removed from our final database

In [29]:
try:
      axis='columns', inplace=True)
except:
    pass

In [30]:
data.columns

Index(['title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'], dtype='object')

In [36]:
print(str(data.iloc[2]['content']))

Image source, Getty Images
Image caption, Industry leaders argue oil and gas extraction cannot be scaled back too quickly
Business leaders have warned that thousands of oil and gas jobs could be at… [+5155 chars]
